# Improve Semantic Similarity with Reverse HYDE

It is common that the documents that we want to retrieve are longer than the users' queries and have different formats. To increase the accuracy of the **r*etrieval of the documents based on the users' queries, we will generate hypothetical potential queries from each document and use them as vector embeddings to the documents - AKA Reverse Hyde.

Please note that the original [Hyde technique](https://arxiv.org/abs/2212.10496) processed the incoming queries of the users, and generated the hypothetical documents from these queries, and then used these hypothetical documents to retrive the real documents. In the reverse HYDE, the processing is done when indexing the documents and not in retrival time. Therefore, the latency of the query is not affectd.

* [Reverse Hyde Implementation](#reverse-hyde-implementation)
* [Enriching Vector Database with Reverse Hyde Output](#enriching-vector-database-with-reverse-hyde-output)
* [Query the Enriched Index](#query-the-enriched-index)

### Visual improvements

We will use [rich library](https://github.com/Textualize/rich) to make the output more readable, and supress warning messages.

In [1]:
from rich.console import Console
from rich_theme_manager import Theme, ThemeManager
import pathlib

theme_dir = pathlib.Path("themes")
theme_manager = ThemeManager(theme_dir=theme_dir)
dark = theme_manager.get("dark")

# Create a console with the dark theme
console = Console(theme=dark)

In [2]:
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

## Reverse HYDE Implementation

We will create a class that will generate and hypotherical questions and also retrieve the document by calculating the semantic similarity matching. In a real application, we can use a vector database for the embedding vector storage, indexing and retrieval. 

In [3]:
import ollama
from typing import List, Dict
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def get_embedding(text: str) -> List[float]:
    """Get embedding for a single text using ollama."""
    response = ollama.embeddings(
        model="nomic-embed-text",  # Best embedding model for ollama
        prompt=text
    )
    return response["embedding"]

def get_embeddings(texts: List[str]) -> List[List[float]]:
    """Get embeddings for multiple texts."""
    return [get_embedding(text) for text in texts]

In [4]:
class ReverseHyde:
    def __init__(self, api_key: str):
        openai.api_key = api_key
        self.model = "text-embedding-ada-002"

    def get_embedding(self, text: str) -> List[float]:
        client = openai.OpenAI()
        response = client.embeddings.create(input=text, model=self.model)
        return response.data[0].embedding

    def generate_reverse_hyde(self, chunk: str, n: int = 3) -> List[str]:
        prompt = f"""
        
Given the following text chunk, generate {n} different questions that this chunk would be a good answer to:

Chunk: {chunk}

Questions (enumarate the questions with 1. 2., etc.):
"""

        client = openai.OpenAI()
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100,
            n=1,
            stop=None,
            temperature=0.7,
        )

        
        questions = response.choices[0].message.content.strip().split('\n')
        return [q.split('. ', 1)[1] for q in questions if '. ' in q]

    def process_chunks(self, chunks: List[str], n: int = 3) -> Dict[str, List[str]]:
        processed_chunks = {}
        for chunk in chunks:
            processed_chunks[chunk] = self.generate_reverse_hyde(chunk, n)
        return processed_chunks


Loading API keys from environment variable

In [5]:
from dotenv import load_dotenv

load_dotenv()

False

## Enriching Vector Database with Reverse Hyde Output

We will apply the Reverse Hyde method on a set of documents, and enrich the vector database index with LLM generated Hypothetical questions.

In [6]:
import ollama
from typing import List, Dict
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

class ReverseHyde:
    def __init__(self):
        self.model = "llama3.2"
        self.embedding_model = "nomic-embed-text"
    
    def generate_questions(self, chunk: str, n: int = 5) -> List[str]:
        """Generate n hypothetical questions for a given chunk."""
        prompt = f"""Generate {n} different questions that could be answered by the following text.
        Return only the questions, one per line, no numbering.
        
        Text: {chunk}
        
        Questions:"""
        
        response = ollama.chat(
            model=self.model,
            messages=[{"role": "user", "content": prompt}]
        )
        
        questions = response["message"]["content"].strip().split("\n")
        return [q.strip() for q in questions if q.strip()][:n]
    
    def get_embedding(self, text: str) -> List[float]:
        """Get embedding for a text."""
        response = ollama.embeddings(
            model=self.embedding_model,
            prompt=text
        )
        return response["embedding"]
    
    def process_chunks(self, chunks: List[str], n: int = 5) -> List[Dict]:
        """Process chunks and generate questions + embeddings."""
        results = []
        for chunk in chunks:
            questions = self.generate_questions(chunk, n)
            chunk_embedding = self.get_embedding(chunk)
            question_embeddings = [self.get_embedding(q) for q in questions]
            
            results.append({
                "chunk": chunk,
                "questions": questions,
                "chunk_embedding": chunk_embedding,
                "question_embeddings": question_embeddings
            })
        return results

# Usage - no API key needed!
reverse_hyde = ReverseHyde()

chunks = [
    "A mitochondrion (pl. mitochondria) is an organelle found in the cells of most eukaryotes, such as animals, plants and fungi. Mitochondria have a double membrane structure and use aerobic respiration to generate adenosine triphosphate (ATP), which is used throughout the cell as a source of chemical energy. They were discovered by Albert von Kölliker in 1857 in the voluntary muscles of insects. Meaning a thread-like granule, the term mitochondrion was coined by Carl Benda in 1898. The mitochondrion is popularly nicknamed the \"powerhouse of the cell\", a phrase popularized by Philip Siekevitz in a 1957 Scientific American article of the same name.",
    "Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation. Python is dynamically typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a \"batteries included\" language due to its comprehensive standard library.",
    "The American Civil War (from April 12, 1861 to May 26, 1865) was a civil war in the United States between the Union (\"the North\") and the Confederacy (\"the South\"), which was formed in 1861 by states that had seceded from the Union. The central conflict leading to war was a dispute over whether slavery should be permitted to expand into the western territories, leading to more slave states, or be prohibited from doing so, which many believed would place slavery on a course of ultimate extinction."
]

processed_chunks = reverse_hyde.process_chunks(chunks, n=5)

In [7]:
console.print(processed_chunks)

[
    {
        'chunk': 'A mitochondrion (pl. mitochondria) is an organelle found in the cells of most eukaryotes, such as
animals, plants and fungi. Mitochondria have a double membrane structure and use aerobic respiration to generate 
adenosine triphosphate (ATP), which is used throughout the cell as a source of chemical energy. They were 
discovered by Albert von Kölliker in 1857 in the voluntary muscles of insects. Meaning a thread-like granule, the 
term mitochondrion was coined by Carl Benda in 1898. The mitochondrion is popularly nicknamed the "powerhouse of 
the cell", a phrase popularized by Philip Siekevitz in a 1957 Scientific American article of the same name.',
        'questions': [
            'What type of cells do most eukaryotes have organelles called mitochondria?',
            'Where were mitochondria first discovered in the scientific community?',
            'Who coined the term "mitochondrion" and what does it mean?',
            'Why is the mitochondrion referred to as the "powerhouse of the cell"?',
            'When was the phrase "powerhouse of the cell" popularized by Philip Siekevitz?'
        ],
        'chunk_embedding': [
            0.7014645338058472,
            1.1923047304153442,
            -3.277801990509033,
            0.16705293953418732,
            1.4418907165527344,
            -0.5613637566566467,
            -1.0914791822433472,
            -0.27365970611572266,
            -0.6217420101165771,
            -0.844342052936554,
            -0.22276313602924347,
            0.45843854546546936,
            2.1895744800567627,
            0.10931050032377243,
            -0.3129090368747711,
            -1.0396853685379028,
            -0.545813798904419,
            -0.8222993612289429,
            0.19782045483589172,
            1.4153192043304443,
            -1.4289162158966064,
            0.585500180721283,
            -0.3292851150035858,
            0.23552367091178894,
            1.2679718732833862,
            1.7406190633773804,
            0.009205532260239124,
            -1.215940237045288,
            0.705232560634613,
            0.6665319204330444,
            1.108288049697876,
            -1.8180617094039917,
            0.5171501636505127,
            0.043582070618867874,
            -0.9814009070396423,
            -0.5605698227882385,
            1.4076910018920898,
            0.6788733601570129,
            0.856751024723053,
            0.37030404806137085,
            -0.1569378674030304,
            -0.6575942039489746,
            -0.5599603056907654,
            -0.8262381553649902,
            0.9335688948631287,
            0.3901481032371521,
            -0.30918174982070923,
            -0.7571603655815125,
            0.6150930523872375,
            -0.5585883259773254,
            0.5307850241661072,
            0.5773874521255493,
            -0.22612588107585907,
            -0.6951234936714172,
            1.4213348627090454,
            0.6396868228912354,
            0.3095409572124481,
            -0.5510515570640564,
            1.0575966835021973,
            0.75606369972229,
            1.3761347532272339,
            1.6540262699127197,
            -0.17390097677707672,
            0.8515893816947937,
            0.6679309606552124,
            -0.9730280041694641,
            -0.3533446192741394,
            1.0370947122573853,
            -0.3024500608444214,
            -0.4017576575279236,
            1.3099998235702515,
            -1.4435468912124634,
            -0.2278328537940979,
            0.7415972352027893,
            0.07535972446203232,
            0.44349682331085205,
            -0.5729317665100098,
            0.3199729025363922,
            0.4538383483886719,
            -0.04032381996512413,
            0.7411770820617676,
            -0.5650466084480286,
            1.4047825336456299,
            -0.24636390805244446,
            1.6444511413574219,
            -0.26689857244491577,
            -0.97310608625

## Query the enriched index

Once we have an index with multiple hypothetical questions to the documents, we can use it to retrive the document based on a real user's query.

In [8]:
query = "What generates energy in a cell?"

In [9]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

# Create the embedding encoder
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [10]:
# Create collection to store the wine rating data
hyde_collection_name="reverse_hyde"

qdrant.recreate_collection(
    collection_name=hyde_collection_name,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

True

In [12]:
import uuid

# vectorize!
qdrant.upload_points(
    collection_name=hyde_collection_name,
    points=[
        models.PointStruct(
            id=uuid.uuid5(uuid.NAMESPACE_URL, f"{d_idx}-{q_idx}").hex,
            vector=encoder.encode(question).tolist(),
            payload={ 
                "document": chunk_data["chunk"],  # Access via dict key
                "doc_id": d_idx
            }
        ) for d_idx, chunk_data 
            in enumerate(processed_chunks)  # iterate list directly
                for q_idx, question in enumerate(chunk_data["questions"])  # Access questions via key
    ]
)

In [13]:
console.print(
    qdrant
    .get_collection(
        collection_name=hyde_collection_name
    )
)

CollectionInfo(
    status=<CollectionStatus.GREEN: 'green'>,
    optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>,
    warnings=None,
    indexed_vectors_count=0,
    points_count=14,
    segments_count=1,
    config=CollectionConfig(
        params=CollectionParams(
            vectors=VectorParams(
                size=384,
                distance=<Distance.COSINE: 'Cosine'>,
                hnsw_config=None,
                quantization_config=None,
                on_disk=None,
                datatype=None,
                multivector_config=None
            ),
            shard_number=None,
            sharding_method=None,
            replication_factor=None,
            write_consistency_factor=None,
            read_fan_out_factor=None,
            read_fan_out_delay_ms=None,
            on_disk_payload=None,
            sparse_vectors=None
        ),
        hnsw_config=HnswConfig(
            m=16,
            ef_construct=100,
            full_scan_threshold=10000,
            max_indexing_threads=0,
            on_disk=None,
            payload_m=None,
            inline_storage=None
        ),
        optimizer_config=OptimizersConfig(
            deleted_threshold=0.2,
            vacuum_min_vector_number=1000,
            default_segment_number=0,
            max_segment_size=None,
            memmap_threshold=None,
            indexing_threshold=20000,
            flush_interval_sec=5,
            max_optimization_threads=1,
            prevent_unoptimized=None
        ),
        wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0, wal_retain_closed=1),
        quantization_config=None,
        strict_mode_config=None,
        metadata=None
    ),
    payload_schema={},
    update_queue=None
)

### Search Collection for Best Match



In [19]:
from rich.panel import Panel
from rich.table import Table

def search_collection(collection_name: str, query: str, limit: int = 1):
    hits = qdrant.query_points(
        collection_name=collection_name,
        query=encoder.encode(query).tolist(),
        limit=limit
    )
    
    # Create a table for both query and best match
    table = Table(show_header=True, header_style="bold yellow")
    table.add_column("Query", style="bright_cyan", width=30)
    table.add_column("Best Matching Chunk", style="bright_yellow", width=50)
    table.add_column("Score", style="bright_green")
    
    for hit in hits.points:  # Access .points
        table.add_row(query, f"{hit.payload['document'][:80]}...", "{:.4f}".format(hit.score))
    
    # Create a panel for the table
    panel = Panel(
        table,
        title=f"[bold]Query and Best Match in {collection_name}",
        border_style="white",
        expand=False
    )
    console.print(panel)

search_collection(hyde_collection_name, query)

╭────────────────────────────── Query and Best Match in reverse_hyde ──────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ │
│ ┃ Query                          ┃ Best Matching Chunk                                ┃ Score  ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ │
│ │ What generates energy in a     │ A mitochondrion (pl. mitochondria) is an organelle │ 0.5422 │ │
│ │ cell?                          │ found in the cells of most eu...                   │        │ │
│ └────────────────────────────────┴────────────────────────────────────────────────────┴────────┘ │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

In [20]:
search_collection(hyde_collection_name, query)

╭────────────────────────────── Query and Best Match in reverse_hyde ──────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ │
│ ┃ Query                          ┃ Best Matching Chunk                                ┃ Score  ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ │
│ │ What generates energy in a     │ A mitochondrion (pl. mitochondria) is an organelle │ 0.5422 │ │
│ │ cell?                          │ found in the cells of most eu...                   │        │ │
│ └────────────────────────────────┴────────────────────────────────────────────────────┴────────┘ │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

### Compare to document only index (without Hyde)

We will index the same documents without adding the reverse Hyde questions and compare the similarity scores.

In [21]:
# Create collection to store the wine rating data
docs_collection_name="documents_only"

qdrant.recreate_collection(
    collection_name=docs_collection_name,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

True

In [24]:
# vectorize!
qdrant.upload_points(
    collection_name=docs_collection_name,
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(chunk_data["chunk"]).tolist(),
            payload={"document": chunk_data["chunk"]}
        ) for idx, chunk_data in enumerate(processed_chunks)  # iterate list directly
    ]
)

In [25]:
search_collection(docs_collection_name, query)

╭───────────────────────────── Query and Best Match in documents_only ─────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ │
│ ┃ Query                          ┃ Best Matching Chunk                                ┃ Score  ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ │
│ │ What generates energy in a     │ A mitochondrion (pl. mitochondria) is an organelle │ 0.5003 │ │
│ │ cell?                          │ found in the cells of most eu...                   │        │ │
│ └────────────────────────────────┴────────────────────────────────────────────────────┴────────┘ │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯